In [1]:
from flask import Flask, request, jsonify
from threading import Thread
import json

In [2]:
app = Flask(__name__)
received_data = []

In [3]:
from madhu import test_run
import nest_asyncio
import asyncio
nest_asyncio.apply()
data="""
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
    "Datetime": "19-07-2025T12:34:55",
    "Location": "IISc Bangalore",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Agentic AI Project Status Update",
    "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
"""
result = await test_run(str(data))
result

Config set: http://localhost:8000/v1


'\n\n{\n  "Priority": "not urugent",\n  "PreferredTime": {\n    "StartTime": "2025-07-24T10:30:00+05:30",\n    "EndTime": "2025-07-24T11:00:00+05:30",\n    "ConflictCheck": {\n      "usertwo.amd@gmail.com": "Busy",\n      "userthree.amd@gmail.com": "Busy"\n    }\n  },\n  "ProposedTime": {\n    "StartTime": "2025-07-24T13:30:00+05:30",\n    "EndTime": "2025-07-24T14:00:00+05:30",\n    "ConflictCheck": {\n      "usertwo.amd@gmail.com": "Available",\n      "userthree.amd@gmail.com": "Available"\n    }\n  },\n  "AlternativeSlots": [\n    {\n      "StartTime": "2025-07-24T14:30:00+05:30",\n      "EndTime": "2025-07-24T15:00:00+05:30"\n    },\n    {\n      "StartTime": "2025-07-24T15:30:00+05:30",\n      "EndTime": "2025-07-24T16:00:00+05:30"\n    }\n  ],\n  "MetaData": {\n    "Subject": "Agentic AI Project Status Update",\n    "Location": "IISc Bangalore",\n    "Organizer": "userone.amd@gmail.com",\n    "Attendees": ["usertwo.amd@gmail.com", "userthree.amd@gmail.com"]\n  },\n  "PoliteEmailD

In [4]:
from dateutil import parser
from madhu import test_run
import nest_asyncio
import asyncio
import json
async def your_meeting_assistant(data): 
    # Your Agentic AI Calls 
    nest_asyncio.apply()
    print("Before agentic ai call")
    result = await test_run(str(data))
    print(" after agentic ai call")
    result_json = json.loads(result)
    print("result", result)
    proposed_time = result_json.get("ProposedTime")
    print("Proposed time as obtained from agentic ai call", proposed_time)
    data["EventStart"] = proposed_time.get("StartTime")
    data["EventEnd"] = proposed_time.get("EndTime")
    start = parser.isoparse(proposed_time.get("StartTime"))
    end = parser.isoparse(proposed_time.get("EndTime"))
    data["Duration_mins"] = str(int((end - start).total_seconds() // 60))
    return data

In [5]:
from datetime import datetime, timedelta
from dateutil import parser
from Calendar_Event_Extraction import retrive_calendar_events
from datetime import timezone

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    print(f"\n Received: {json.dumps(data, indent=2)}")

    # Ensure the "From" email is part of Attendees
    from_email = data.get("From")
    attendees = data.get("Attendees", [])
    if from_email and not any(att.get("email") == from_email for att in attendees):
        attendees.append({"email": from_email})
    data["Attendees"] = attendees

    # Call agentic assistant to populate EventStart/End
    updated_data = asyncio.run(your_meeting_assistant(data))
    try:
        start_date = parser.isoparse(updated_data["EventStart"]).date()
        end_date = parser.isoparse(updated_data["EventEnd"]).date()
        print(start_date, type(start_date), end_date, type(end_date))
    except Exception as e:
        return jsonify({"error": f"Invalid datetime format: {e}"}), 400

    # start_dt = datetime.combine(start_date, datetime.min.time()).isoformat() + "+05:30"
    # end_dt = datetime.combine(end_date, datetime.max.time()).isoformat() + "+05:30"

    IST = timezone(timedelta(hours=5, minutes=30))
    start_dt = datetime.combine(start_date, datetime.min.time(), tzinfo=IST).isoformat()
    end_dt = datetime.combine(end_date, datetime.max.time(), tzinfo=IST).isoformat()

  

    print(f"Should print all events of all users from {start_dt} to {end_dt}")
    print(type(start_dt), type(end_dt))

    # Append events to each attendee
    for attendee in updated_data["Attendees"]:
        email = attendee.get("email")
        if email:
            events = retrive_calendar_events(email, str(start_dt), str(end_dt))
            attendee["Events"] = events

    received_data.append(updated_data)
    print(f"\n\n\n Sending:\n {json.dumps(updated_data, indent=2)}")
    return jsonify(updated_data)


def run_flask():
    app.run(host='0.0.0.0', port=5000)

In [6]:
email="userthree.amd@gmail.com"
start_dt="2025-07-24T00:00:00+05:30"
end_dt="2025-07-24T23:59:59+05:30"
print(type(start_dt), type(end_dt))
retrive_calendar_events(email, start_dt, end_dt)

<class 'str'> <class 'str'>


[{'StartTime': '2025-07-23T16:00:00+05:30',
  'EndTime': '2025-07-24T07:30:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'},
 {'StartTime': '2025-07-24T10:30:00+05:30',
  'EndTime': '2025-07-24T11:00:00+05:30',
  'NumAttendees': 3,
  'Attendees': ['userthree.amd@gmail.com',
   'userone.amd@gmail.com',
   'usertwo.amd@gmail.com'],
  'Summary': 'Agentic AI Project Status Update'},
 {'StartTime': '2025-07-24T16:00:00+05:30',
  'EndTime': '2025-07-25T07:30:00+05:30',
  'NumAttendees': 1,
  'Attendees': ['SELF'],
  'Summary': 'Off Hours'}]

In [7]:
# Start Flask in a background thread
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://129.212.191.89:5000
Press CTRL+C to quit



 Received: {
  "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
  "Datetime": "19-07-2025T12:34:55",
  "Location": "IISc Bangalore",
  "From": "userone.amd@gmail.com",
  "Attendees": [
    {
      "email": "usertwo.amd@gmail.com"
    },
    {
      "email": "userthree.amd@gmail.com"
    }
  ],
  "Subject": "Agentic AI Project Status Update",
  "EmailContent": "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."
}
Before agentic ai call


[2025-07-20 07:32:26,921] ERROR in app: Exception on /receive [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/httpcore/_async/connection.py", line 101, in handle_async_request
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/htt